Install dependencies

In [ ]:
!apt-get update
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
# install required system dependencies
!apt-get install -y xvfb x11-utils

# install required python dependencies (might need to install additional gym extras depending)
!pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [ ]:
import os
import sys
import shutil

Running_in_colab = 'google.colab' in sys.modules

if Running_in_colab:
  sys_name = 'COLAB'
else:
  sys_name = os.environ['COMPUTERNAME']

# Importing Libs

if not os.path.exists("lib"):
    print("LOADING FILES")

    if not os.path.exists("lib.zip"):
        !wget -q -O lib.zip https://www.dropbox.com/s/k9trf8pgs8dsyxg/lib.zip?dl=0
        
    if not os.path.exists("lib"):
        !unzip "/content/lib.zip" -d "/content"
        # file_names = os.listdir('/content/lib')
        # for file_name in file_names:
        #     print(file_name)
        #     shutil.move(os.path.join(
        #         '/content/lib', file_name), '/content/')

LOADING FILES
Archive:  /content/lib.zip
   creating: /content/lib/
  inflating: /content/lib/archs.py   
 extracting: /content/lib/desktop.ini  
  inflating: /content/lib/my_tools.py  
  inflating: /content/lib/performance_evaluation.py  
  inflating: /content/lib/train_loop.py  
 extracting: /content/lib/__init__.py  


In [ ]:
#@title Imports
import numpy as np
import matplotlib.pyplot as plt
import time
from matplotlib.pyplot import figure

import gym
import torch
from torch.utils import data
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import random
from gym import wrappers
import copy
import gym
from gym.wrappers import Monitor
import glob
import io
import base64

from IPython.core.display import HTML
from IPython.core.display import Video
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
import sys
Running_in_colab = 'google.colab' in sys.modules

display = Display(visible=0, size=(1400, 900))
display.start()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

from lib import archs as archs
from lib import performance_evaluation as eval
from lib import train_loop as tl
from lib import my_tools as mt
# !mkdir Checkpoints
import time


In [ ]:
#@title Deep RL Replay Buffer

class ReplayBuffer:
    def __init__(self, size, input_shape,dev):
        self.size = size
        self.counter = 0
        self.state_buffer = torch.zeros((self.size, input_shape))
        self.action_buffer = torch.zeros(self.size, dtype=torch.int64)
        self.reward_buffer = torch.zeros(self.size)
        self.new_state_buffer = torch.zeros((self.size, input_shape))
        self.terminal_buffer = torch.zeros(self.size)
        self.state_buffer.to(dev)
        self.action_buffer.to(dev)
        self.reward_buffer.to(dev)
        self.new_state_buffer.to(dev)
        self.terminal_buffer.to(dev)
        self.dev=dev;



    def store_tuples(self, state, action, reward, new_state, done):
        idx = self.counter % self.size
        self.state_buffer[idx] = state
        self.action_buffer[idx] = action
        self.reward_buffer[idx] = reward
        self.new_state_buffer[idx] = new_state
        self.terminal_buffer[idx] = done
        self.counter += 1

    def sample_buffer(self, batch_size):
        max_buffer = min(self.counter, self.size)
        batch = np.random.choice(max_buffer, batch_size, replace=False)
        state_batch = self.state_buffer[batch].to(self.dev)
        action_batch = self.action_buffer[batch].to(self.dev)
        reward_batch = self.reward_buffer[batch].to(self.dev)
        new_state_batch = self.new_state_buffer[batch].to(self.dev)
        done_batch = self.terminal_buffer[batch].to(self.dev)

        return state_batch, action_batch, reward_batch, new_state_batch, done_batch
    def last_buffer(self, batch_size):
        state_batch = self.state_buffer[-1:-batch_size].to(self.dev)
        action_batch = self.action_buffer[-1:-batch_size].to(self.dev)
        reward_batch = self.reward_buffer[-1:-batch_size].to(self.dev)
        new_state_batch = self.new_state_buffer[-1:-batch_size].to(self.dev)
        done_batch = self.terminal_buffer[-1:-batch_size].to(self.dev)

        return state_batch, action_batch, reward_batch, new_state_batch, done_batch


In [ ]:
#@title DQN movie generation (for visual evaluation in Google Colab)

def createMovie(Network,path,Filename):
    env = gym.make("CartPole-v0")
    envX = wrappers.Monitor(env,path+'/'+Filename,force=True)

    observation = envX.reset()

    i=0
    Network.eval()

    while True:
        envX.render()
        
        state=torch.Tensor(observation).to(device)
      
        action = Network.policy(state);
             
        observation, reward, done, info = envX.step(action.item()) 
        i=i+1;     
        if done: 
          break;

    envX.close()
    env.close();
    Network.train()
    mp4list = glob.glob(path+Filename+'/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        if(Running_in_colab):
            ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded.decode('ascii'))))
        else:
            Video(mp4)
    else: 
        print("Could not find video")

Initialization and Parameters:


In [ ]:
# Sim configuration
env = gym.make("CartPole-v0")
spec = gym.spec("CartPole-v0")



inputs=4
n_actions=2

#hyper-parameters
TotalEpisodes=2000
MaxSteps=400
FreezeCounter=25
BatchSize=128
exploration_threshold=1
exploration_threshold_min=0.01
# exploration_decay=0.002
discount_factor=0.99
SaveAtCounter=200
LearningRateDecay=0.99


#network DQN

buffer = ReplayBuffer(1000000, inputs,device);

for arch in ["DQN","DuelingDQN","DoubleDQN","DoubleDuelingDQN"]:
    file_path__ = mt.create_dir("results",arch)
    for n_layers in [4]:
        policy_net, target_net = archs.archs(arch,inputs,n_actions,discount_factor,device,n_layers)
        target_net.load_state_dict(policy_net.state_dict())
        target_net.eval()
        file_path_ = mt.create_dir(file_path__,"N"+str(n_layers)+"_Layers")
        if(arch == "DQN"):
          lrlist = [0.001]
          expldecay = [0.002]
        if(arch == "DuelingDQN"):
          lrlist = [1e-5]
          expldecay = [0.002]
        else:
          lrlist = [1e-4]
          expldecay = [0.02]
        for LearningRate in lrlist:
            for exploration_decay in expldecay: 
                optimizer = torch.optim.Adam(policy_net.parameters(), lr=LearningRate)
                scheduler = optim.lr_scheduler.ExponentialLR(optimizer,gamma=LearningRateDecay)
                loss = torch.nn.MSELoss()
                def trainModel():
                    if buffer.counter < BatchSize:
                        return 0.0
                    
                    state_batch, action_batch, reward_batch, new_state_batch, done_batch = buffer.sample_buffer(BatchSize)

                    q_actual = torch.gather(policy_net(state_batch),1,action_batch.reshape(-1,1))
                    with torch.no_grad():
                        if(arch == "DoubleDQN" or arch == "DoubleDuelingDQN"):
                            target = torch.argmax(policy_net(state_batch), -1).detach()
                            q_max_next = target_net(new_state_batch).gather(1, target.unsqueeze(-1)).squeeze(-1)
                        else:
                            q_max_next = target_net(new_state_batch).max(1)[0].detach()
                    q_target = (q_max_next * discount_factor)*(1-done_batch) + reward_batch

                    ll=loss(q_actual, q_target.unsqueeze(1))

                    # Optimize the model
                    optimizer.zero_grad()
                    ll.backward()
                    for param in policy_net.parameters():
                        param.grad.data.clamp_(-1, 1)
                    optimizer.step()

                    return ll.item();
                file_name = mt.set_name(arch,n_layers,BatchSize,exploration_threshold,exploration_threshold_min,exploration_decay,discount_factor,LearningRate,LearningRateDecay)
                file_path = mt.create_dir(file_path_,file_name)
                mt.create_dir(file_path,"Checkpoints")
                print(file_name)
                bestNet, episodes, scores, events, avg_scores, avg_scores20, exploration, avg_scores100 = tl.train_loop(policy_net, target_net, env, device, TotalEpisodes, FreezeCounter, SaveAtCounter, createMovie, MaxSteps, exploration_threshold, exploration_decay, exploration_threshold_min, buffer, trainModel,file_path)

                eval.performance_evaluation(file_path,episodes, scores, events, avg_scores, avg_scores20, exploration, avg_scores100)
                eval.report(file_path,arch,BatchSize,exploration_threshold,exploration_threshold_min,exploration_decay,discount_factor,
                            LearningRate,LearningRateDecay,episodes, scores, events, avg_scores, avg_scores20, exploration, n_layers,avg_scores100)
                createMovie(bestNet,file_path,'bestNet')
                time.sleep(1)

300 of 2000


KeyboardInterrupt: ignored